# PROYECTO FINAL - modificación de columnas

In [3]:
# Tratamiento de datos
import numpy as np
import pandas as pd
#import sidetable as stb

# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
import sidetable as stb

# Asunciones y Preprocesamiento
from scipy import stats
import math
from scipy.stats import levene
#import researchpy as rp
from sklearn.preprocessing import StandardScaler
import itertools
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols
import researchpy as rp # chi square

#Linear Regresion con Sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = False

In [4]:
df_bike = pd.read_csv("./bikes.csv", index_col=0)
df_bike.sample(6)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
370,371,06-01-2019,spring,1,1,0,5,1,1,13.700847,17.01335,54.2083,11.249836,307,3791,4098
391,392,27-01-2019,spring,1,1,0,5,1,2,17.425000,20.76915,74.1250,22.958689,269,3187,3456
303,304,31-10-2018,winter,0,10,0,1,1,1,13.940000,17.80315,70.3333,7.125450,362,3307,3669
210,211,30-07-2018,autumn,0,7,0,6,0,1,32.970847,36.42685,46.5833,11.291979,1559,2916,4475
453,454,30-03-2019,summer,1,3,0,5,1,2,15.170000,18.78105,58.0833,9.250489,796,4663,5459
329,330,26-11-2018,winter,0,11,0,6,0,1,15.409153,19.25435,68.1667,4.584194,1249,1819,3068


# MODIFICACIONES EN NUESTRAS COLUMNAS:

# 1. Fecha:
- Tenemos "yr" (año) y "mnth" (mes) y obtenemos "day" a través de la separación de "dteday", que posteriormente eliminaremos:

In [5]:
df_bike["day"] = df_bike["dteday"].str.split(pat= "-", n = -1, expand = True)[0]

In [6]:
df_bike=df_bike.reindex(columns=['instant', 'dteday', 'season', 'yr', 'mnth','day', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'])

In [7]:
df_bike.head()

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,01,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,02,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,03,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,04,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,05,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


------

# 2. Columna "season"
- Corregir de acuerdo con las fechas.

In [8]:
df_bike['dteday'] = df_bike['dteday'].apply(pd.to_datetime)

In [9]:
def meses(x):
    a = x.month

    if  a >= 1 and a<=3:
        return 'winter'
    elif a >= 4 and a<=6:
        return 'spring'
    elif a >= 7 and a<=9:
        return 'summer'
    else:
        return 'autumn'

In [10]:
df_bike['season'] = df_bike['dteday'].apply(meses)

In [11]:
df_bike.sample(5)

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
61,62,2018-03-03,winter,0,3,03,0,4,1,1,8.131653,10.00665,31.8333,15.125518,123,1562,1685
442,443,2019-03-19,winter,1,3,19,0,1,1,1,22.345000,26.64105,72.8750,10.875239,982,5171,6153
262,263,2018-09-20,summer,0,9,20,0,2,1,2,23.028347,26.61085,88.1250,9.041918,438,3203,3641
425,426,2019-02-03,winter,1,3,02,0,5,1,2,14.486653,17.99210,65.7083,9.708568,246,2948,3194
494,495,2019-10-05,autumn,1,5,10,0,4,1,1,20.739153,24.58915,55.2083,21.042221,1026,5546,6572


 - Encoding numérico para la columna season

In [12]:
scale_mapper = {"winter":1, "spring":1, "summer":2, 'autumn':3}

df_bike['season_encoding'] = df_bike["season"].replace(scale_mapper)

--------

# 8. "weathersit": 
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

In [13]:
weather = pd.DataFrame(df_bike[["cnt", "weathersit"]].groupby("weathersit"))

In [14]:
i = 0
lista2 = []
for i in range(len(df_bike["weathersit"].value_counts())):
    a = weather[1][i]["cnt"].sum()
    i = i+1
    lista2.append(a)

In [15]:
weathersit = pd.DataFrame(lista2)
weathersit["%"] = weathersit[0].apply(lambda x: x / df_bike.shape[0] * 100)
weathersit["Tipo"] = ["1", "2", "3"]

In [16]:
labelsX = ["1: Clear, Few clouds, Partly cloudy, Partly cloudy", "2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist", "Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds"]


In [17]:
df_bike.head(2)

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_encoding
0,1,2018-01-01,winter,0,1,01,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,1
1,2,2018-02-01,winter,0,1,02,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,1


---

# 9. Temperatura, Sensación Térmica y Humedad

### Hay que hacer encoding con etiquetas para las graficas

In [18]:
df_bike.head(2)

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_encoding
0,1,2018-01-01,winter,0,1,01,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,1
1,2,2018-02-01,winter,0,1,02,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,1


In [19]:
df_bike[['temp','atemp','hum']] = df_bike[['temp','atemp','hum']].round(1)

###  Codificando variables

In [20]:
df_bike["temp_interval"] = pd.cut(df_bike["temp"], 5,labels = ["very_low", "low", "medium", "high", "very_high"], ordered=True)

In [21]:
df_bike["temp_interval_num"] = pd.cut(df_bike["temp"], 5, ordered=True)

In [22]:
df_bike["hum_interval"] = pd.cut(df_bike["hum"], 5,labels = ["very_low", "low", "medium", "high", "very_high"], ordered=True)

In [23]:
df_bike["hum_interval_num"] = pd.cut(df_bike["hum"], 5, ordered=True)

### Voy a hacer un ordinal encoding

In [24]:
from sklearn.preprocessing import OrdinalEncoder
orden = ["very_low", "low", "medium", "high", "very_high"]

In [25]:
def ordinal_encoder(df, columna, orden_valores):
    
    # nos creamos un diccionario vacío para hacer el map después. 
    ordinal_dict = {}
    
    # iteramos por nuestra lista de valores usando el enumerate que recordamos nos devolvía también el índice o posición de cada elemento
    for i, valor in enumerate(orden_valores):
        ordinal_dict[valor]=i+1 # le sumamos uno a la posición para no tener valores de 0. 
    
    # aplicamos el map
    df[columna+"_ord"] = df[columna].map(ordinal_dict)
    return df

In [26]:
ordinal_encoder(df_bike, "temp_interval", orden)

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_encoding,temp_interval,temp_interval_num,hum_interval,hum_interval_num,temp_interval_ord
0,1,2018-01-01,winter,0,1,01,0,6,0,2,14.1,18.2,80.6,10.749882,331,654,985,1,low,"(8.98, 15.56]",very_high,"(77.76, 97.2]",2
1,2,2018-02-01,winter,0,1,02,0,0,0,2,14.9,17.7,69.6,16.652113,131,670,801,1,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2
2,3,2018-03-01,winter,0,1,03,0,1,1,1,8.1,9.5,43.7,16.636703,120,1229,1349,1,very_low,"(2.367, 8.98]",medium,"(38.88, 58.32]",1
3,4,2018-04-01,spring,0,1,04,0,2,1,1,8.2,10.6,59.0,10.739832,108,1454,1562,1,very_low,"(2.367, 8.98]",high,"(58.32, 77.76]",1
4,5,2018-05-01,spring,0,1,05,0,3,1,1,9.3,11.5,43.7,12.522300,82,1518,1600,1,low,"(8.98, 15.56]",medium,"(38.88, 58.32]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,726,2019-12-27,autumn,1,12,27,0,4,1,2,10.4,11.3,65.3,23.458911,247,1867,2114,3,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2
726,727,2019-12-28,autumn,1,12,28,0,5,1,2,10.4,12.8,59.0,10.416557,644,2451,3095,3,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2
727,728,2019-12-29,autumn,1,12,29,0,6,0,2,10.4,12.1,75.3,8.333661,159,1182,1341,3,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2
728,729,2019-12-30,autumn,1,12,30,0,0,0,1,10.5,11.6,48.3,23.500518,364,1432,1796,3,low,"(8.98, 15.56]",medium,"(38.88, 58.32]",2


In [27]:
ordinal_encoder(df_bike, "hum_interval", orden)

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_encoding,temp_interval,temp_interval_num,hum_interval,hum_interval_num,temp_interval_ord,hum_interval_ord
0,1,2018-01-01,winter,0,1,01,0,6,0,2,14.1,18.2,80.6,10.749882,331,654,985,1,low,"(8.98, 15.56]",very_high,"(77.76, 97.2]",2,5
1,2,2018-02-01,winter,0,1,02,0,0,0,2,14.9,17.7,69.6,16.652113,131,670,801,1,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2,4
2,3,2018-03-01,winter,0,1,03,0,1,1,1,8.1,9.5,43.7,16.636703,120,1229,1349,1,very_low,"(2.367, 8.98]",medium,"(38.88, 58.32]",1,3
3,4,2018-04-01,spring,0,1,04,0,2,1,1,8.2,10.6,59.0,10.739832,108,1454,1562,1,very_low,"(2.367, 8.98]",high,"(58.32, 77.76]",1,4
4,5,2018-05-01,spring,0,1,05,0,3,1,1,9.3,11.5,43.7,12.522300,82,1518,1600,1,low,"(8.98, 15.56]",medium,"(38.88, 58.32]",2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,726,2019-12-27,autumn,1,12,27,0,4,1,2,10.4,11.3,65.3,23.458911,247,1867,2114,3,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2,4
726,727,2019-12-28,autumn,1,12,28,0,5,1,2,10.4,12.8,59.0,10.416557,644,2451,3095,3,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2,4
727,728,2019-12-29,autumn,1,12,29,0,6,0,2,10.4,12.1,75.3,8.333661,159,1182,1341,3,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2,4
728,729,2019-12-30,autumn,1,12,30,0,0,0,1,10.5,11.6,48.3,23.500518,364,1432,1796,3,low,"(8.98, 15.56]",medium,"(38.88, 58.32]",2,3


---

# 10. "Windspeed": encoding

### Se ponen etiquetas a viento

In [28]:
df_bike['wind_interval'] = pd.cut(df_bike['windspeed'], 5, labels=["very_low", "low", "medium", "high", "very_high"], ordered=True)

In [29]:
df_bike['wind_interval'] = df_bike['wind_interval'].astype('object')

### Encoding numérico para la columna wind

In [30]:
scale_mapper = {"very low":0, "low":1, "medium":2, 'high':3, 'very high':4}

df_bike['wind_num_encoding'] = df_bike["wind_interval"].replace(scale_mapper)

In [31]:
df_bike.head()

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_encoding,temp_interval,temp_interval_num,hum_interval,hum_interval_num,temp_interval_ord,hum_interval_ord,wind_interval,wind_num_encoding
0,1,2018-01-01,winter,0,1,01,0,6,0,2,14.1,18.2,80.6,10.749882,331,654,985,1,low,"(8.98, 15.56]",very_high,"(77.76, 97.2]",2,5,low,1
1,2,2018-02-01,winter,0,1,02,0,0,0,2,14.9,17.7,69.6,16.652113,131,670,801,1,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2,4,medium,2
2,3,2018-03-01,winter,0,1,03,0,1,1,1,8.1,9.5,43.7,16.636703,120,1229,1349,1,very_low,"(2.367, 8.98]",medium,"(38.88, 58.32]",1,3,medium,2
3,4,2018-04-01,spring,0,1,04,0,2,1,1,8.2,10.6,59.0,10.739832,108,1454,1562,1,very_low,"(2.367, 8.98]",high,"(58.32, 77.76]",1,4,low,1
4,5,2018-05-01,spring,0,1,05,0,3,1,1,9.3,11.5,43.7,12.522300,82,1518,1600,1,low,"(8.98, 15.56]",medium,"(38.88, 58.32]",2,3,low,1


In [32]:
df_bike.to_csv('bike_columnas_modificadas.cvs')